In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from google.colab import files
from ultralytics import YOLO
from moviepy.editor import VideoFileClip

# Load YOLOv8 model for vehicle detection
model = YOLO('yolov8s.pt')

# Upload video from local machine
uploaded = files.upload()
video_path = next(iter(uploaded))  # Get the uploaded video filename

# Set up video capture
video = cv2.VideoCapture(video_path)

# Define the output video settings
output_path = 'processed_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
    ret, or_frame = video.read()
    
    # Restart video if it ends
    if not ret:
        break

    # Apply Gaussian blur to the frame
    frame = cv2.GaussianBlur(or_frame, (5, 5), 0)
    
    # Convert frame to HSV for lane detection mask
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_y = np.array([18, 94, 140])
    upper_y = np.array([48, 255, 255])
    mask = cv2.inRange(hsv, lower_y, upper_y)
    edges = cv2.Canny(mask, 74, 150)
    
    # Detect lane lines using Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, maxLineGap=50)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green for lane lines

    # Vehicle detection
    results = model(or_frame)
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf.item()
        cls = int(box.cls.item())

        # Only display detection if it's a car and confidence is above threshold
        if model.names[cls] == 'car' and conf >= 0.5:
            label = f'{model.names[cls]} {conf:.2f}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)  # Yellow for cars
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    # Write the processed frame to output video
    out.write(frame)

# Release resources
video.release()
out.release()

# Display the output video
from IPython.display import Video
Video(output_path, embed=True)
